In [1]:
from strands import Agent

# Create an agent with the default model (Claude 3.7 Sonnet on Bedrock)
agent = Agent()

# This is equivalent to:
agent = Agent(model="us.anthropic.claude-3-7-sonnet-20250219-v1:0")

In [2]:
# Ask the agent a complex multi-part question involving reasoning, computation, and real-time awareness
response = agent(
"What is 1234 multiplied by 5678, what is the square root of 1444"
)

I'll solve these two problems:

1) 1234 multiplied by 5678:
   1234 × 5678 = 7,006,652

2) Square root of 1444:
   √1444 = 38

The multiplication gives us 7,006,652, and the square root of 1444 is exactly 38.

In [3]:
import boto3
from strands import Agent
from strands.models import BedrockModel

# Create a BedrockModel with custom configuration
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    region_name='us-east-1',
    temperature=0.3,  # Lower temperature for more deterministic outputs
    max_tokens=1024,  # Limit response length
    # Optional: custom Bedrock client
    client=boto3.client('bedrock-runtime', region_name='us-east-1')
)

# Create an agent with the custom model configuration
agent = Agent(model=bedrock_model)  

In [4]:
response = agent(
"Convert 100°F to Celsius, and then convert 5 kilometers to miles."
)

# Converting 100°F to Celsius

To convert from Fahrenheit to Celsius, I'll use the formula:
°C = (°F - 32) × 5/9

°C = (100 - 32) × 5/9
°C = 68 × 5/9
°C = 340/9
°C = 37.78°C

So 100°F equals 37.78°C.

# Converting 5 kilometers to miles

To convert kilometers to miles, I'll use the conversion factor:
1 kilometer = 0.621371 miles

5 kilometers = 5 × 0.621371 miles
5 kilometers = 3.107 miles

So 5 kilometers equals 3.107 miles.

In [5]:
from strands import tool

@tool
def my_custom_tool(param1: str, param2: int = 0) -> str:
    """Tool description that helps the agent understand when to use this tool.
    
    Args:
        param1: Description of the first parameter
        param2: Description of the second parameter with a default value
        
    Returns:
        Description of what the tool returns
    """
    # Tool implementation
    result = f"Processed {param1} with value {param2}"
    return result

In [9]:
from strands import Agent, tool
import requests

@tool
def get_weather(location: str) -> str:
    """Get current weather information for a location.
    
    Args:
        location: City name or location (e.g., 'Seattle', 'New York')
        
    Returns:
        Current weather information including temperature and conditions
    """
    # Note: In a real implementation, you would use a proper weather API
    # This is a simplified example
    try:
        # Using a free weather API (replace with your preferred API)
        api_key = "idontknowkey"  # Get from environment variable in production
        url = f"https://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}&units=metric"
        
        response = requests.get(url)
        data = response.json()
        
        if response.status_code == 200:
            temp = data["main"]["temp"]
            condition = data["weather"][0]["description"]
            humidity = data["main"]["humidity"]
            
            return f"Weather in {location}: {condition}, {temp}°C, Humidity: {humidity}%"
        else:
            return f"Error fetching weather: {data.get('message', 'Unknown error')}"
    except Exception as e:
        return f"Failed to get weather information: {str(e)}"

# Create an agent with our custom tool
agent = Agent(
    tools=[get_weather],
    system_prompt="You are a helpful assistant that can provide weather information."
)

# Ask the agent about the weather
response = agent("What's the weather like in Chennai now?")
print(response.message)

I'll check the current weather in Chennai for you.
Tool #1: get_weather
The current weather in Chennai is overcast clouds with a temperature of 32.69°C. The humidity is at 62%.{'role': 'assistant', 'content': [{'text': 'The current weather in Chennai is overcast clouds with a temperature of 32.69°C. The humidity is at 62%.'}]}


In [13]:
from strands import Agent, tool
import requests
import json
from typing import Dict, List, Optional, Union
from functools import lru_cache
import os
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
# Define tools for US stock market data using Alpha Vantage API
@tool
def search_us_stocks(query: str, limit: int = 5) -> List[Dict]:
    """Search for US stocks matching the query.
    
    Use this tool when the user is looking for US stock information.
    The search matches partial company names and ticker symbols.
    
    Args:
        query: Search term for company name or ticker symbol
        limit: Maximum number of results to return (default: 5)
        
    Returns:
        List of matching stocks with their details
    """
    try:
        api_key = os.getenv("ALPHA_VANTAGE_API_KEY")
        url = f"https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords={query}&apikey={api_key}"
        
        response = requests.get(url)
        data = response.json()
        
        if "bestMatches" not in data:
            return f"Error: Unable to find US stocks matching '{query}'. API response: {data}"
        
        results = []
        for match in data["bestMatches"][:limit]:
            results.append({
                "symbol": match["1. symbol"],
                "name": match["2. name"],
                "type": match["3. type"],
                "region": match["4. region"],
                "currency": match["8. currency"],
                "market": "US"
            })
        
        return results
    except ConnectionError:
        return "Unable to connect to Alpha Vantage API. Please check your internet connection."
    except Exception as e:
        return f"Error searching US stocks: {str(e)}"


In [16]:
@tool
def get_us_stock_price(ticker: str) -> Dict:
    """Get the current price and information for a US stock ticker.
    
    Args:
        ticker: Stock ticker symbol (e.g., 'AAPL', 'MSFT')
        
    Returns:
        Current stock price information and company details
    """
    try:
        api_key = os.getenv("ALPHA_VANTAGE_API_KEY")
        url = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={ticker}&apikey={api_key}"
        
        response = requests.get(url)
        quote_data = response.json()
        
        if "Global Quote" not in quote_data or not quote_data["Global Quote"]:
            return f"Invalid ticker symbol: {ticker}. Please provide a valid US stock symbol."
        
        # Get company overview for additional information
        overview_url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey={api_key}"
        overview_response = requests.get(overview_url)
        overview_data = overview_response.json()
        
        quote = quote_data["Global Quote"]
        
        result = {
            "symbol": ticker,
            "price": quote["05. price"],
            "change": quote["09. change"],
            "change_percent": quote["10. change percent"],
            "volume": quote["06. volume"],
            "market": "US"
        }
        
        # Add company information if available
        if "Name" in overview_data:
            result["name"] = overview_data["Name"]
            result["sector"] = overview_data["Sector"]
            result["industry"] = overview_data["Industry"]
            result["description"] = overview_data["Description"]
        
        return result
    except ConnectionError:
        return "Unable to connect to Alpha Vantage API. Please check your internet connection."
    except Exception as e:
        return f"Error fetching US stock price: {str(e)}"

In [ ]:
@tool
def analyze_financial_stock(ticker: str, market: str = "US", exchange: str = "NSE") -> Dict:
    """Analyze a financial stock and provide key metrics.
    
    Args:
        ticker: Stock ticker symbol
        market: Market to search in ("US" or "India")
        exchange: Exchange code for Indian stocks (default: 'NSE', can be 'BSE')
        
    Returns:
        Dictionary with financial analysis and recommendations
    """
    try:
        if market.lower() == "us":
            # For US stocks, use Alpha Vantage
            api_key = os.getenv("ALPHA_VANTAGE_API_KEY")
            
            # Get company overview
            overview_url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey={api_key}"
            overview_response = requests.get(overview_url)
            overview_data = overview_response.json()
            
            if not overview_data or "Symbol" not in overview_data:
                return f"Invalid ticker symbol: {ticker}. Please provide a valid US stock symbol."
            
            # Get key metrics
            analysis = {
                "symbol": ticker,
                "name": overview_data["Name"],
                "sector": overview_data["Sector"],
                "industry": overview_data["Industry"],
                "pe_ratio": overview_data["PERatio"],
                "peg_ratio": overview_data["PEGRatio"],
                "dividend_yield": overview_data["DividendYield"],
                "eps": overview_data["EPS"],
                "52_week_high": overview_data["52WeekHigh"],
                "52_week_low": overview_data["52WeekLow"],
                "market_cap": overview_data["MarketCapitalization"],
                "profit_margin": overview_data["ProfitMargin"],
                "beta": overview_data["Beta"],
                "market": "US"
            }
            
            return analysis
    except Exception as e:
        return f"Error analyzing financial stock: {str(e)}"


In [22]:
# Create the agent with our financial tools
financial_agent = Agent(
    # Using Claude 3.5 Sonnet model from Bedrock
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    
    # Add our financial tools
    tools=[
        search_us_stocks,
        get_us_stock_price,
        analyze_financial_stock
    ],
    
    # Customize the system prompt for financial analysis
    system_prompt="""You are a helpful financial assistant specializing in stock market analysis.
    
    You can search for and analyze stocks from both US and Indian markets.
    
    When providing stock information:
    1. Always specify which market (US or India) the stock belongs to
    2. Include key financial metrics when available
    3. Explain what the metrics mean in simple terms
    4. Provide context about the company's industry and sector 
    5) If the API tools fail for Indian stocks - Check on Fallback
    For financial analysis:
    - PE Ratio: Explain if it's high or low compared to industry average
    - Dividend Yield: Note if it's attractive for income investors
    - Market Cap: Categorize as large, mid, or small cap
    - Recent Performance: Note significant price movement
    
    Always be transparent about the limitations of your analysis and remind users
    that this information should not be considered financial advice. Provide the results in Table format
    """
)

In [23]:
response = financial_agent("Find technology stocks in the US market with good growth potential")
print(response.message)

I'd be happy to help you find some technology stocks in the US market with good growth potential. Let me search for some technology-focused stocks for you.
Tool #14: search_us_stocks
Let me search for some major tech stocks that would be more suitable for individual stock analysis:
Tool #15: search_us_stocks
Let me try searching for them individually:
Tool #16: search_us_stocks
Now I'll analyze several key technology stocks to find ones with good growth potential. Let's start with Microsoft:
Tool #17: analyze_financial_stock
Let me check Apple:
Tool #18: search_us_stocks

Tool #19: analyze_financial_stock
Let's check NVIDIA:
Tool #20: search_us_stocks

Tool #21: analyze_financial_stock
Let's also check Amazon:
Tool #22: search_us_stocks

Tool #23: analyze_financial_stock
Let's check Google/Alphabet:
Tool #24: search_us_stocks

Tool #25: analyze_financial_stock
# US Technology Stocks with Growth Potential

Based on my analysis of major US technology stocks, here's a detailed comparison 